In [4]:
import os
import pickle
import time
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
import warnings

warnings.filterwarnings('ignore')

def prepOneHotEncoder(df, col, pathPackages):
    oneHotEncoder = OneHotEncoder(handle_unknown='ignore')
    dfOneHotEncoder = pd.DataFrame(oneHotEncoder.fit_transform(df[[col]]).toarray(),
                                   columns=[col + "_" + str(i+1) for i in range(len(df[col].unique()))])
    
    filename = 'prep' + col + '.pkl'
    pickle.dump(oneHotEncoder, open(pathPackages + filename, 'wb'))
    print(f"Preprocessing data {col} has been saved...")
    
    df = pd.concat([df.drop(col, axis=1), dfOneHotEncoder], axis=1)
    return df

def prepStandardScaler(df, col, pathPackages):
    scaler = StandardScaler()
    df[col] = scaler.fit_transform(df[[col]])
    
    filename = 'prep' + col + '.pkl'
    pickle.dump(scaler, open(pathPackages + filename, 'wb'))
    print(f"Preprocessing data {col} has been saved...")
    
    return df

def runModel(data, path):
    path = os.path.join(path, "modelling", "packages")
    col = pickle.load(open(os.path.join(path, 'columnModelling.pkl'), 'rb'))
    df = pd.DataFrame(data, index=[0])
    df = df[col]

    prepType = pickle.load(open(os.path.join(path, 'preptype.pkl'), 'rb'))
    dfType = pd.DataFrame(prepType.transform(df[['type']]).toarray(),
                          columns=["type_" + str(i+1) for i in range(len(prepType.transform(df[['type']]).toarray()[0]))])
    df = pd.concat([df.drop('type', axis=1), dfType], axis=1)

    X = df.values.tolist()
    model = pickle.load(open(os.path.join(path, 'modelFraud.pkl'), 'rb'))
    y = model.predict(X)[0]
    if y == 0:
        return "White List"
    else:
        return "Fraud"

if __name__ == "__main__":
    pathPackages = os.getcwd() + "\\" + "packages" + "\\"
    target = 'isFraud'
    
    data = pd.read_csv(pathPackages + 'Fraud_Detection.csv')
    data = data.drop(['nameOrig', 'nameDest'], axis=1)
    
    df = data.drop(target, axis=1)
    pickle.dump(df.columns.tolist(), open(pathPackages + 'columnModelling.pkl', 'wb'))

    colOneHotEncoder = ['type']
    for col in colOneHotEncoder:
        df = prepOneHotEncoder(df, col, pathPackages)

    colprepStandardScaler = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
    for col in colprepStandardScaler:
        df = prepStandardScaler(df, col, pathPackages)

    X = df.values.tolist()
    y = data[[target]].values.ravel()
    
    start = time.time()
    
    model = LogisticRegression()
    model.fit(X, y)
    stop = time.time()
    
    with open(pathPackages + 'modelFraud.pkl', 'wb') as file:
        pickle.dump(model, file)
    print(f"Training model done in {stop-start} seconds...")

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\RickyS-PC\\Desktop\\Training\\Tech-software\\Bootcamp\\Digital-Skola-FS-Data-Engineer\\Class\\mini-project4\\modelling\\packages\\Fraud_Detection.csv'